In [1]:
import numpy as np
import pandas as pd

In [2]:
attributes = pd.read_excel('DIAS Attributes - Values 2017.xlsx').drop('Unnamed: 0', axis=1)
attribute_info = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx').drop('Unnamed: 0', axis=1)

In [3]:
attributes[['Attribute', 'Description']] = attributes[['Attribute', 'Description']].ffill()

In [4]:
attribute_info['Information level'] = attribute_info['Information level'].ffill()

In [5]:
attributes.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly


In [6]:
def mark_as_missing(x):
    if type(x)==str:
        return 'unknown' in x
    else:
        return x
    
attributes['Missing'] = attributes['Meaning'].apply(mark_as_missing)

In [7]:
attribute_info.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,Person,ANREDE_KZ,gender,NaN
3,Person,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,Person,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [8]:
cols_TYP = attribute_info.loc[attribute_info.Attribute.str.contains('TYP'), :].Attribute.unique()
cols_KLASSE = attribute_info.loc[attribute_info.Attribute.str.contains('KLASSE'), :].Attribute.unique()
cols_categoricalA = attribute_info.loc[attribute_info.Description.str.contains('classif'), :].Attribute.unique()
cols_typology = attribute_info.loc[attribute_info.Description.str.contains('typology'), :].Attribute.unique()

cols_typ = attributes.loc[attributes.Attribute.str.contains('TYP'), :].Attribute.unique()
cols_klasse = attributes.loc[attributes.Attribute.str.contains('KLASSE'), :].Attribute.unique()

cols_categoricalB = attributes.loc[attributes.Description.str.contains('classif'), :].Attribute.unique()
cols_class = attributes.loc[attributes.Description.str.contains('typology'), :].Attribute.unique()

In [10]:
attribute_info[attribute_info.Attribute == 'LP_FAMILIE_GROB']

,Information level,Attribute,Description,Additional notes
18,Person,LP_FAMILIE_GROB,family type rough,NaN


In [2]:
attributes = pd.read_pickle('attributes.pkl')

In [17]:
attributes.Meaning.unique()

array(['unknown', 'no classification possible', 'passive elderly',
       'cultural elderly', 'experience-driven elderly', '< 30 years',
       '30 - 45 years', '46 - 60 years', '> 60 years',
       'uniformly distributed', 'unknown / no main age detectable',
       '01.01.1895 bis 31.12.1899', '01.01.1900 bis 31.12.1904',
       '01.01.1905 bis 31.12.1909', '01.01.1910 bis 31.12.1914',
       '01.01.1915 bis 31.12.1919', '01.01.1920 bis 31.12.1924',
       '01.01.1925 bis 31.12.1929', '01.01.1930 bis 31.12.1934',
       '01.01.1935 bis 31.12.1939', '01.01.1940 bis 31.12.1944',
       '01.01.1945 bis 31.12.1949', '01.01.1950 bis 31.12.1954',
       '01.01.1955 bis 31.12.1959', '01.01.1960 bis 31.12.1964',
       '01.01.1965 bis 31.12.1969', '01.01.1970 bis 31.12.1974',
       '01.01.1975 bis 31.12.1979', '01.01.1980 bis 31.12.1984',
       '01.01.1985 bis 31.12.1989', '01.01.1990 bis 31.12.1994',
       '01.01.1995 bis 31.12.1999', 'male', 'female',
       'numeric value (typically cod

In [5]:
categorical_attributes_info = attributes.loc[attributes.Value.apply(lambda x: (type(x)==str) & (x!='-1, 0') & (x!='-1, 9')), :]
categorical_attributes_info.reset_index(inplace=True, drop=True)

In [6]:
categorical_attributes_info.head()

,Attribute,Description,Value,Meaning,Missing
0,ANZ_HAUSHALTE_AKTIV,number of households in the building,…,numeric value (typically coded from 1-10),False
1,ANZ_HH_TITEL,number of academic title holder in building,…,numeric value (typically coded from 1-10),False
2,ANZ_PERSONEN,number of adult persons in the household,…,numeric value (typically coded from 1-3),False
3,ANZ_TITEL,number of professional title holder in household,…,numeric value (typically coded from 1-10),False
4,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1A,Work-Life-Balance,False


In [7]:
categorical_attributes_info = categorical_attributes_info[categorical_attributes_info.Value!='…']
categorical_attributes_info.head()

,Attribute,Description,Value,Meaning,Missing
4,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1A,Work-Life-Balance,False
5,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1B,Wealthy Best Ager,False
6,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1C,Successful Songwriter,False
7,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1D,Old Nobility,False
8,CAMEO_DEU_2015,CAMEO classification 2015 - detailled classifi...,1E,City Nobility,False


In [8]:
binary_attrib = attributes.query("Missing == False").groupby('Attribute').size()
binary_attrib_index = binary_attrib[binary_attrib == 2].index

In [9]:
binary_attributes_info = attributes.loc[attributes.Attribute.apply(lambda x: x in binary_attrib_index), :].query("Missing == False")
binary_attributes_info.reset_index(inplace=True, drop=True)

In [10]:
binary_attributes_info.head()

,Attribute,Description,Value,Meaning,Missing
0,ANREDE_KZ,gender,1,male,False
1,ANREDE_KZ,gender,2,female,False
2,BIP_FLAG,business-flag indicating companies in the buil...,0,no company in the building,False
3,BIP_FLAG,business-flag indicating companies in the buil...,1,company in the building,False
4,GREEN_AVANTGARDE,Green avantgarde,0,doesn't belong to the green avantgarde,False


In [11]:
typ_attributes = attributes.loc[attributes.Attribute.str.contains('TYP'), :]
klasse_attributes = attributes.loc[attributes.Attribute.str.contains('KLASSE'), :]
    
attributes_typ_descr = attributes.loc[attributes.Description.str.contains('typ'), :]
attributes_class_descr = attributes.loc[attributes.Description.str.contains('class'), :]    

In [12]:
set(list(categorical_attributes_info.Attribute.unique()) + list(binary_attributes_info.Attribute.unique()))

{'ANREDE_KZ',
 'BIP_FLAG',
 'CAMEO_DEU_2015',
 'GREEN_AVANTGARDE',
 'KBA05_SEG6',
 'ORTSGR_KLS9',
 'OST_WEST_KZ',
 'SOHO_FLAG',
 'VERS_TYP'}

In [13]:
attributes_cat = pd.concat([categorical_attributes_info, binary_attributes_info, typ_attributes, klasse_attributes, attributes_typ_descr, attributes_class_descr], axis=0)

In [14]:
len(attributes_cat.Attribute.unique())

47

In [16]:
attributes_cat.Attribute.unique()

array(['CAMEO_DEU_2015', 'ORTSGR_KLS9', 'OST_WEST_KZ', 'ANREDE_KZ',
       'BIP_FLAG', 'GREEN_AVANTGARDE', 'KBA05_SEG6', 'SOHO_FLAG',
       'VERS_TYP', 'AGER_TYP', 'CJT_GESAMTTYP', 'D19_KONSUMTYP',
       'D19_KK_KUNDENTYP', 'FINANZTYP', 'GEBAEUDETYP',
       'GEBAEUDETYP_RASTER', 'GFK_URLAUBERTYP', 'HEALTH_TYP', 'REGIOTYP',
       'RETOURTYP_BK_S', 'SHOPPER_TYP', 'ZABEOTYP',
       'KBA13_SEG_KOMPAKTKLASSE', 'KBA13_SEG_MITTELKLASSE',
       'KBA13_SEG_OBEREMITTELKLASSE', 'KBA13_SEG_OBERKLASSE',
       'CAMEO_DEUINTL_2015', 'FINANZ_ANLEGER', 'FINANZ_HAUSBAUER',
       'FINANZ_MINIMALIST', 'FINANZ_SPARER', 'FINANZ_UNAUFFAELLIGER',
       'FINANZ_VORSORGER', 'KBA05_BAUMAX', 'LP_FAMILIE_FEIN',
       'LP_FAMILIE_GROB', 'PLZ8_BAUMAX', 'ALTERSKATEGORIE_GROB',
       'CAMEO_DEUG_2015', 'KBA05_KRSOBER', 'KBA05_MOD1', 'KBA05_MOD2',
       'KBA05_MOD3', 'KBA05_SEG3', 'KBA05_SEG4', 'KBA05_SEG5',
       'KBA13_KRSSEG_OBER'], dtype=object)

In [4]:
attributes[attributes.Description=='familytyp fine']

,Attribute,Description,Value,Meaning,Missing
1892,LP_FAMILIE_FEIN,familytyp fine,1,single,False
1893,LP_FAMILIE_FEIN,familytyp fine,2,couple,False
1894,LP_FAMILIE_FEIN,familytyp fine,3,young single parent,False
1895,LP_FAMILIE_FEIN,familytyp fine,4,single parent with teenager,False
1896,LP_FAMILIE_FEIN,familytyp fine,5,single parent with child of full age,False
1897,LP_FAMILIE_FEIN,familytyp fine,6,young family,False
1898,LP_FAMILIE_FEIN,familytyp fine,7,family with teenager,False
1899,LP_FAMILIE_FEIN,familytyp fine,8,family with child of full age,False
1900,LP_FAMILIE_FEIN,familytyp fine,9,shared flat,False
1901,LP_FAMILIE_FEIN,familytyp fine,10,two-generational household,False


In [61]:
attributes.to_pickle('attributes.pkl')